In [1]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pyarrow as pa

### Info for PyArrow Install https://www.delftstack.com/howto/python/read-and-write-parquet-files-in-python/

In [2]:
# Create a dictionary.
data = {}
# Loop through the tables.
for tableNumber in range(1, 6):
    # Assign the path to a variable to use when accessing the file.
    table_import_path = f'Resources/flight_data_{tableNumber}.parquet'
    # Load the data frame.
    df = pd.read_parquet(table_import_path)
    # Make boolean data numeric.
    df["isBasicEconomy"] = df["isBasicEconomy"].replace("True", 1).replace("False", 0)
    df["isRefundable"] = df["isRefundable"].replace("True", 1).replace("False", 0)
    df["isNonStop"] = df["isNonStop"].replace("True", 1).replace("False", 0)
    # Make dates and 'travelDuration' object data into datetime
    df['searchDate'] = pd.to_datetime(df['searchDate'])
    df['flightDate'] = pd.to_datetime(df['flightDate'])
    df['travelDuration'] = pd.to_timedelta(df['travelDuration'])
    # Add new column for how many days before a flight a search took place.
    df["searchDaysBeforeFlight"] = pd.to_datetime(df["flightDate"]) - pd.to_datetime(df["searchDate"])
    # Remove some columns.
    df.drop(columns=['legId','fareBasisCode','elapsedDays','baseFare','seatsRemaining',
                     'segmentsDepartureTimeEpochSeconds','segmentsDepartureTimeRaw',
                     'segmentsArrivalTimeEpochSeconds','segmentsArrivalTimeRaw','segmentsArrivalAirportCode',
                     'segmentsDepartureAirportCode','segmentsAirlineName','segmentsAirlineCode',
                     'segmentsEquipmentDescription','segmentsDurationInSeconds',
                     'segmentsDistance','segmentsCabinCode'], inplace=True, errors='ignore')
    # Assign the path to a variable to use when accessing the file.
    table_export_path = f'Resources/flight_data_clean_{tableNumber}.csv'
    # Export table to csv.
    df.to_csv(table_export_path)
    # Copy the dataframe to the dictionary.
    data[tableNumber] = df
# Join the tables.
df = pd.concat([data[1], data[2], data[3], data[4], data[5]], axis=0)
# Recalculate the row numbers.
df = df.reset_index()
del df['index']

In [3]:
# Assign the path to a variable to use when accessing the file.
table_export_path = f'Resources/flight_data_clean_all.csv'
# Export table to csv.
df.to_csv(table_export_path)

In [4]:
df = pd.DataFrame(df)
df.head()

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,searchDaysBeforeFlight
0,2022-09-28,2022-10-05,SFO,ORD,0 days 07:05:00,0,0,0,451.60,1933,7 days
1,2022-09-28,2022-10-05,SFO,ORD,0 days 07:52:00,0,0,0,451.60,1933,7 days
2,2022-09-28,2022-10-05,SFO,ORD,0 days 08:34:00,0,0,0,451.60,1933,7 days
3,2022-09-28,2022-10-05,SFO,PHL,0 days 07:49:00,0,0,0,103.99,2590,7 days
4,2022-09-28,2022-10-05,SFO,PHL,0 days 13:19:00,0,0,0,109.59,2590,7 days


# Clean Data

Essential data cleaning methods to consider when first interacting with a new dataset:

**Check for missing values**: Check the dataset for any missing values and decide on a strategy to handle them, such as imputing the missing values or removing the affected rows/columns.

**Check for duplicates**: Check the dataset for any duplicate records and decide on a strategy to handle them, such as removing the duplicates or aggregating them.

**Check for data types**: Check the data types of each column in the dataset to ensure they are appropriate for the data they represent. For example, dates should be represented as dates and not as strings.

**Check for outliers**: Check for any outliers or extreme values in the dataset that may skew your analysis, and decide on a strategy to handle them, such as removing them or replacing them with more appropriate values.

**Check for inconsistencies**: Check for inconsistencies or errors in the data, such as typos or formatting issues, and decide on a strategy to handle them, such as cleaning the data or removing the affected records.

**Normalize or scale the data**: If you have numerical data that varies widely in magnitude, consider normalizing or scaling the data so that the values are comparable.

**Check for data quality**: Check the overall quality of the data, such as the accuracy and completeness of the information, to ensure that the data is suitable for analysis.

(Thanks ChatGPT)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752040 entries, 0 to 1752039
Data columns (total 11 columns):
 #   Column                  Dtype          
---  ------                  -----          
 0   searchDate              datetime64[ns] 
 1   flightDate              datetime64[ns] 
 2   startingAirport         object         
 3   destinationAirport      object         
 4   travelDuration          timedelta64[ns]
 5   isBasicEconomy          int64          
 6   isRefundable            int64          
 7   isNonStop               int64          
 8   totalFare               object         
 9   totalTravelDistance     object         
 10  searchDaysBeforeFlight  timedelta64[ns]
dtypes: datetime64[ns](2), int64(3), object(4), timedelta64[ns](2)
memory usage: 147.0+ MB


In [6]:
df.shape

(1752040, 11)

In [7]:
df.describe()

,travelDuration,isBasicEconomy,isRefundable,isNonStop,searchDaysBeforeFlight
count,1752040,1.752040e+06,1.752040e+06,1.752040e+06,1752040
mean,0 days 07:20:15.812275975,2.678090e-01,4.566106e-06,2.503778e-01,23 days 12:07:29.644528663
std,0 days 03:52:09.334555272,4.428176e-01,2.136841e-03,4.332308e-01,12 days 18:54:46.872849528
min,0 days 00:59:00,0.000000e+00,0.000000e+00,0.000000e+00,1 days 00:00:00
25%,0 days 04:32:00,0.000000e+00,0.000000e+00,0.000000e+00,13 days 00:00:00
50%,0 days 06:59:00,0.000000e+00,0.000000e+00,0.000000e+00,24 days 00:00:00
75%,0 days 09:38:00,1.000000e+00,0.000000e+00,1.000000e+00,34 days 00:00:00
max,1 days 08:39:00,1.000000e+00,1.000000e+00,1.000000e+00,49 days 00:00:00


In [8]:
df.isnull().count()

searchDate                1752040
flightDate                1752040
startingAirport           1752040
destinationAirport        1752040
travelDuration            1752040
isBasicEconomy            1752040
isRefundable              1752040
isNonStop                 1752040
totalFare                 1752040
totalTravelDistance       1752040
searchDaysBeforeFlight    1752040
dtype: int64

In [9]:
for col in df.columns:
    unique_values = df[col].unique()
    print(f"{col} column: {unique_values}")

searchDate column: ['2022-09-28T00:00:00.000000000' '2022-09-29T00:00:00.000000000'
 '2022-09-30T00:00:00.000000000' '2022-10-01T00:00:00.000000000']
flightDate column: ['2022-10-05T00:00:00.000000000' '2022-10-06T00:00:00.000000000'
 '2022-10-07T00:00:00.000000000' '2022-10-08T00:00:00.000000000'
 '2022-10-09T00:00:00.000000000' '2022-10-10T00:00:00.000000000'
 '2022-10-11T00:00:00.000000000' '2022-10-12T00:00:00.000000000'
 '2022-10-13T00:00:00.000000000' '2022-10-14T00:00:00.000000000'
 '2022-10-15T00:00:00.000000000' '2022-10-16T00:00:00.000000000'
 '2022-10-17T00:00:00.000000000' '2022-10-18T00:00:00.000000000'
 '2022-10-19T00:00:00.000000000' '2022-10-20T00:00:00.000000000'
 '2022-10-21T00:00:00.000000000' '2022-10-22T00:00:00.000000000'
 '2022-10-23T00:00:00.000000000' '2022-10-24T00:00:00.000000000'
 '2022-10-25T00:00:00.000000000' '2022-10-26T00:00:00.000000000'
 '2022-10-27T00:00:00.000000000' '2022-10-28T00:00:00.000000000'
 '2022-10-29T00:00:00.000000000' '2022-10-30T00:00:

# Converting Duration Column to proper DateTime Format

In [10]:
# Turn travelDuration column into DateTime
df['travelDuration'] = pd.to_timedelta(df['travelDuration'])
df

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,searchDaysBeforeFlight
0,2022-09-28,2022-10-05,SFO,ORD,0 days 07:05:00,0,0,0,451.60,1933,7 days
1,2022-09-28,2022-10-05,SFO,ORD,0 days 07:52:00,0,0,0,451.60,1933,7 days
2,2022-09-28,2022-10-05,SFO,ORD,0 days 08:34:00,0,0,0,451.60,1933,7 days
3,2022-09-28,2022-10-05,SFO,PHL,0 days 07:49:00,0,0,0,103.99,2590,7 days
4,2022-09-28,2022-10-05,SFO,PHL,0 days 13:19:00,0,0,0,109.59,2590,7 days
...,...,...,...,...,...,...,...,...,...,...,...
1752035,2022-10-01,2022-11-09,CLT,LGA,0 days 04:45:00,1,0,0,119.10,548,39 days
1752036,2022-10-01,2022-11-09,CLT,LGA,0 days 04:49:00,1,0,0,119.10,705,39 days
1752037,2022-10-01,2022-11-09,CLT,LGA,0 days 04:59:00,1,0,0,119.10,548,39 days
1752038,2022-10-01,2022-11-09,CLT,LGA,0 days 05:06:00,1,0,0,119.10,560,39 days
